In [1]:
# from queryFuncs import _top_n_by_column, top_n_by_column
# import os
# from Methods.base import similarity_filter
# data = top_n_by_column(2023, "GeoPro", 100000, "/media/nguyen/VIS_DB/DBs/VN_7Fs/METHOD_0/f.db")

In [2]:
# list_column = ["GeoPro", "GeoNgn", "GeoNgn2"]
# list_year = [i for i in range(2016, 2024)]
# for year in list_year:
#     os.makedirs(f"Compare_VN_JP/Data_CT/{year}", exist_ok=True)
#     for col in list_column:
#         data = top_n_by_column(year, col, 100000, "/media/nguyen/VIS_DB/DBs/VN_7Fs/METHOD_0/f.db")
#         df = similarity_filter(data, "CT", 100, 1)
#         df.to_csv(f"Compare_VN_JP/Data_CT/{year}/{col}.csv", index=False)

In [3]:
import pandas as pd
import numpy as np
from Methods.base import calculate_formula, Base, convert_strF_to_arrF
import getValueFuncs as gvf

data_VN_ = pd.read_excel("File_7_truong_HOSE_File3_2023_Field.xlsx")
data_JP_ = pd.read_excel("Compare_VN_JP/File_7_field_Year_Japan_File3.xlsx")

def compare(df_CT, year):
    if type(df_CT) == str:
        df_CT = pd.read_csv(df_CT)

    data_VN = data_VN_[data_VN_["TIME"] <= year].reset_index(drop=True)
    data_VN = data_VN[data_VN["TIME"] >= 2007]
    vis = Base(data_VN)

    data_JP = data_JP_[data_JP_["TIME"] <= year].reset_index(drop=True)
    data_JP = data_JP[data_JP["TIME"] >= 2013]
    vis_JP = Base(data_JP)

    list_value = []
    list_value_JP = []
    for ct in df_CT.CT:
        temp = [ct]
        temp_JP = []
        ct = convert_strF_to_arrF(ct)
        weight = calculate_formula(ct, vis.OPERAND)
        weight_JP = calculate_formula(ct, vis_JP.OPERAND)

        temp.extend(list(gvf.single_investment(
            weight, vis.INDEX, vis.PROFIT, vis.PROFIT_RANK, vis.SYMBOL, 1.06, 1
        )[0]))
        temp.extend(list(gvf.multi_investment(
            weight, vis.INDEX, vis.PROFIT, vis.PROFIT_RANK, vis.SYMBOL, 1.06, 1
        )[0]))
        temp.extend(list(gvf.multi_investment_strictly(
            weight, vis.INDEX, vis.PROFIT, vis.PROFIT_RANK, vis.SYMBOL, 1.06, 1
        )[0]))
        temp.extend(list(gvf.multi_investment_skip_20p_lowprofit(
            weight, vis.INDEX, vis.PROFIT, vis.PROFIT_RANK, vis.SYMBOL, 1.06, 1
        )[0]))
        temp.extend(list(gvf.multi_investment_skip_negative_profit(
            weight, vis.INDEX, vis.PROFIT, vis.PROFIT_RANK, vis.SYMBOL, 1.06, 1
        )[0]))
        list_value.append(temp)

        temp_JP.extend(list(gvf.single_investment(
            weight_JP, vis_JP.INDEX, vis_JP.PROFIT, vis_JP.PROFIT_RANK, vis_JP.SYMBOL, 1.06, 1
        )[0]))
        temp_JP.extend(list(gvf.multi_investment(
            weight_JP, vis_JP.INDEX, vis_JP.PROFIT, vis_JP.PROFIT_RANK, vis_JP.SYMBOL, 1.06, 1
        )[0]))
        temp_JP.extend(list(gvf.multi_investment_strictly(
            weight_JP, vis_JP.INDEX, vis_JP.PROFIT, vis_JP.PROFIT_RANK, vis_JP.SYMBOL, 1.06, 1
        )[0]))
        temp_JP.extend(list(gvf.multi_investment_skip_20p_lowprofit(
            weight_JP, vis_JP.INDEX, vis_JP.PROFIT, vis_JP.PROFIT_RANK, vis_JP.SYMBOL, 1.06, 1
        )[0]))
        temp_JP.extend(list(gvf.multi_investment_skip_negative_profit(
            weight_JP, vis_JP.INDEX, vis_JP.PROFIT, vis_JP.PROFIT_RANK, vis_JP.SYMBOL, 1.06, 1
        )[0]))
        list_value_JP.append(temp_JP)

    list_col = ["CT"] + "GeoPro, HarPro, Value, Profit, ValGLim, GeoLim, ValHLim, HarLim, GeoRank, HarRank".split(", ")\
                      + "Nguong, GeoNgn, HarNgn, ProNgn".split(", ")\
                      + "Nguong2, GeoNgn2, HarNgn2, ProNgn2".split(", ")\
                      + "Nguong_20, GeoNgn_20, HarNgn_20, ProNgn_20".split(", ")\
                      + "Nguong_snp, GeoNgn_snp, HarNgn_snp, ProNgn_snp".split(", ")

    df_VN = pd.DataFrame(list_value)
    df_VN.columns = list_col
    df_JP = pd.DataFrame(list_value_JP)
    df_JP.columns = [col+"_JP" for col in list_col[1:]]
    del vis
    del vis_JP
    k = len(df_VN.columns)
    for col in df_JP.columns[::-1]:
        df_VN.insert(k, column=col, value=df_JP[col])
        k -= 1

    return df_VN

In [4]:
k = 0
for col in ["GeoPro", "GeoNgn", "GeoNgn2"]:
    for year in range(2016, 2023):
        df_CT = pd.read_csv(f"Compare_VN_JP/Data_CT/{year}/{col}.csv")
        data = compare(df_CT, year)
        data.to_csv(f"Compare_VN_JP/Data_CT/{year}/{col}_compared.csv", index=False)